In [5]:
import requests
import time
from tqdm import tqdm
import json

API_KEY = 'RGAPI-1a1d24de-0002-4894-85cc-6deaf6ec560e'
DIVISION = ['I' , 'II' , 'III' , 'IV']
PAGE_NUM = range(50)

def getPuuid(summonerId):
    return requests.get(f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={API_KEY}').json()["puuid"]
        
def getSummonerId(puuid):
    return requests.get(f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={API_KEY}').json()['id']
    
def getMatchIds(puuid):
    return requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={API_KEY}').json()

def getMatchData(matchId):
    return requests.get(f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key={API_KEY}").json()
      
def getLeague(rank , division = None , page_num = None):    # rank == {'challenger' , 'grandmaster' , 'master' , 'diamond' , 'platinum' , 'gold' , 'silver' , 'bronze' , 'iron'}  \\  DIVISION = ['I' , 'II' , 'III' , 'IV']    \\    PAGE_NUM = range(50)

    if rank in ['challenger' , 'grandmaster' , 'master']:   # 마스터 이상의 경우에는 별도의 api 로 호출한다.
        return requests.get(f'https://kr.api.riotgames.com/lol/league/v4/{rank}leagues/by-queue/RANKED_SOLO_5x5?api_key={API_KEY}').json()

    else:   # 다이아 이하 랭크
        return requests.get(f'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank.upper()}/{division}?page={page_num}&api_key={API_KEY}').json()

def saveJson(data , matchid , division = '' , page_num = ''):
    with open(f'{matchid}_{division}_{page_num}.json', 'w') as f:
        json.dump(data, f)


#### MASTER LEAGUE 이상 (!! 얘는 패스)

In [3]:
#### 각 랭크별 경기 데이터 수집.

# 수집할 랭크들
ranks = ['challenger' , 'grandmaster' , 'master'] 

for rank in ranks:

    leagueData = getLeague(rank)
    saveJson(leagueData, rank) # 당시 league 내의 유저들 정보 저장

    entries = getLeague(rank)['entries']
    for entry in tqdm(entries):
        try:
            matchIds = getMatchIds(getPuuid(entry['summonerId']))
            for matchId in matchIds:
                matchData = getMatchData(matchId)
                saveJson(matchData, matchId)
        except:
            pass

 16%|█▌        | 1819/11464 [26:16:38<138:01:57, 51.52s/it]

#### MASTER LEAGUE 미만 (!! 얘를 실행하세요.)

In [ ]:
#### 각 랭크별 경기 데이터 수집.

# 수집할 랭크들
ranks = ['diamond' , 'platinum' , 'gold' , 'silver' , 'bronze' , 'iron'] 

for rank in ranks:

    leagueData = getLeague(rank)
    saveJson(leagueData, rank) # 당시 league 내의 유저들 정보 저장

    for div in DIVISION:
        for page_n in PAGE_NUM:
            entries = getLeague(rank , division = div , page_num = page_n)
            for entry in tqdm(entries):
                try:
                    matchIds = getMatchIds(getPuuid(entry['summonerId']))
                    for matchId in matchIds:
                        matchData = getMatchData(matchId)
                        saveJson(matchData, matchid = matchId , division = div , page_num = page_n)
                except:
                    pass